In [33]:
import pandas as pd
import pulp as lp
from pulp import *

In [34]:
# Reading the data from the excel sheets.
source_df = pd.read_excel("Source_facility_info.xlsx", sheet_name= "PZA")
destination_df = pd.read_excel("Destination_facility_info.xlsx", sheet_name= "PZE")
trucking_df = pd.read_excel("Trucking_info.xlsx", sheet_name= "Truck")

# Convert relevant columns to datetime# Convert start_shift and end_shift to numerical hours and normalize end_shift to handle the next day
def normalize_shift_times(row):
    start_hour = row['Start of shift'].hour + row['Start of shift'].minute / 60
    end_hour = row['End of lay-on'].hour + row['End of lay-on'].minute / 60
    if end_hour < start_hour:
        end_hour += 24  # normalize end_hour for the next day
    return pd.Series([start_hour, end_hour])

destination_df[['Start of shift', 'End of lay-on']] = destination_df.apply(normalize_shift_times, axis=1)
source_df['planned_end_of_loading'] = pd.to_datetime(source_df['planned_end_of_loading'])

In [35]:
# Initialize the model
model = lp.LpProblem("DHL_Optimization", lp.LpMinimize)

# Defining variables
source_list = list(trucking_df['Origin_ID'].unique()[:3])  # Index of PZA
destination_list = list(trucking_df['Destination_ID'].unique()[:3])  # Index of PZE
routes_list = [(i, j) for i in source_list for j in destination_list if i != j]
consignment_list =  [x
    for (i, j) in routes_list
    for x in source_df[(source_df['Origin_ID'] == i) & (source_df['Destination_ID'] == j)]['id'].values
]
valid_combinations = [(i, j, k) for (i, j) in routes_list for k in consignment_list if k in source_df[(source_df['Origin_ID'] == i) & (source_df['Destination_ID'] == j)]['id'].values]

trucks = range(100)

# Decision Variables
# X - Whether a truck goes from I to J
X = lp.LpVariable.dicts("X", [(i, j, k , l) for (i, j, k) in valid_combinations for l in trucks], cat='Binary')

# # Y - Consignment K being carried by K or not
# Y = lp.LpVariable.dicts("Y", [(k, l) for k in source_df['id'].unique() for l in trucks], cat='Binary')

# Z - From a pool of Trucks, whether a truck is used or not
Z = lp.LpVariable.dicts("Z", trucks, cat='Binary')

# T - Time of departure of a given truck
T = lp.LpVariable.dicts("T", trucks, lowBound=0, cat='Integer')

ArrivalDay = lp.LpVariable.dicts("ArrivalDay", trucks, lowBound=0, cat='Integer')  # 0 for same day, 1 for next day, etc.

ArrivalTime = lp.LpVariable.dicts("ArrivalTime", [(i, j, k, l) for (i, j, k) in valid_combinations for l in trucks], lowBound=0, upBound=24, cat='Continuous')

In [36]:
# Combining the objective function: Maximizing the (E+1)th day output and minimizing the distance
model += lp.lpSum(ArrivalDay[l] for l in trucks)

In [37]:
# Constraints
# 1. Each truck can carry at most 2 consignments
for l in trucks:
    model += lp.lpSum([X[(i, j, k, l)] for (i, j, k) in valid_combinations]) <= 2 * Z[l]

In [38]:
# 2. Consignment can only be released after the latest release time of the consignments
for (i, j, k) in valid_combinations:
    release_time = source_df[source_df['id'] == k]['planned_end_of_loading'].dt.hour.values[0]  # Convert to hours
    for l in trucks:
        model += T[l] >= release_time * X[(i, j, k, l)]

In [39]:
# 3. Truck must arrive at the destination within the operational hours

for (i, j, k) in valid_combinations:
    start_shift = destination_df[destination_df['Destination_ID'] == j]['Start of shift'].values[0]
    end_shift = destination_df[destination_df['Destination_ID'] == j]['End of lay-on'].values[0]
    travel_time = trucking_df[(trucking_df['Origin_ID'] == i) & (trucking_df['Destination_ID'] == j)]['OSRM_time [sek]'].values[0] / 3600  # Convert to hours
    for l in trucks:
        # Auxiliary variable to handle modulo operation
        multiplier = lp.LpVariable(f"multiplier_{i}_{j}_{k}_{l}", cat='Integer')
        model += ArrivalTime[(i, j, k, l)] == (T[l] + travel_time * X[(i, j, k, l)] + 24 * ArrivalDay[l]) - 24 * multiplier
        model += ArrivalTime[(i, j, k, l)] >= start_shift
        model += ArrivalTime[(i, j, k, l)] <= end_shift

In [40]:
# 4. Each consignment must be assigned to exactly one truck
for k in consignment_list:
    model += lp.lpSum([X[(i, j, k, l)] for (i, j) in routes_list for l in trucks if (i, j, k) in valid_combinations]) == 1

In [31]:
# 5. Flow conservation: If a truck leaves a source, it must go to one destination
for l in trucks:
    for i in source_list:
        for k in consignment_list:
            model += lp.lpSum([X[(i, j, k, l)] for j in destination_list if j != i  if (i, j, k) in valid_combinations]) == Z[l]

In [41]:
#6. Sorting Capacity: Each PZE should have enough capacity to accomodate all the incoming trucks
# date_chosen = '2024-04-16'
destination_df['Working hours'] = destination_df['End of lay-on'] - destination_df['Start of shift']
for j in destination_list:
    model += lp.lpSum([X[(i, j, k, l)] * source_df[source_df['id'] == k]['Consignment quantity'].values[0]
                      for i in source_list if j != i for k in consignment_list for l in trucks if (i, j, k) in valid_combinations]) <= destination_df[destination_df['Destination_ID'] == j]['Working hours'].values[0] * destination_df[destination_df['Destination_ID'] == j]['Sorting capacity'].values[0]

In [42]:
# Solve the model
model.solve()

In [12]:
value(model.objective)

0.0

In [14]:
solution = {}
for var in model.variables():
    if var.varValue == 1:
        solution[var.name] = var.varValue
print(solution)

{"X_('01.1.1.PZ',_'04.1.1.PZ',_7791768,_97)": 1.0, "X_('01.1.1.PZ',_'04.1.1.PZ',_7791769,_97)": 1.0, "X_('01.1.1.PZ',_'04.1.1.PZ',_7791773,_58)": 1.0, "X_('01.1.1.PZ',_'04.1.1.PZ',_7801226,_33)": 1.0, "X_('01.1.1.PZ',_'04.1.1.PZ',_7801227,_16)": 1.0, "X_('01.1.1.PZ',_'04.1.1.PZ',_7801229,_1)": 1.0, "X_('01.1.1.PZ',_'04.1.1.PZ',_7916247,_97)": 1.0, "X_('01.1.1.PZ',_'04.1.1.PZ',_7916248,_97)": 1.0, "X_('01.1.1.PZ',_'04.1.1.PZ',_7916250,_50)": 1.0, "X_('01.1.1.PZ',_'04.1.1.PZ',_7920338,_5)": 1.0, "X_('01.1.1.PZ',_'04.1.1.PZ',_7942917,_22)": 1.0, "X_('01.1.1.PZ',_'04.1.1.PZ',_7942917,_94)": 1.0, "X_('01.1.1.PZ',_'04.1.1.PZ',_7942917,_97)": 1.0, "X_('01.1.1.PZ',_'04.1.1.PZ',_7942918,_32)": 1.0, "X_('01.1.1.PZ',_'04.1.1.PZ',_7942919,_44)": 1.0, "X_('01.1.1.PZ',_'04.1.1.PZ',_7943959,_67)": 1.0, "X_('01.1.1.PZ',_'04.1.1.PZ',_7943960,_64)": 1.0, "X_('01.1.1.PZ',_'08.1.1.PZ',_7791652,_67)": 1.0, "X_('01.1.1.PZ',_'08.1.1.PZ',_7791655,_30)": 1.0, "X_('01.1.1.PZ',_'08.1.1.PZ',_7801292,_53)": 1.0, "